In [ ]:
# Standard data science libraries
import psutil
import humanize
import os
from IPython.display import display_html

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./"))
dataDirectory= "./" 
print(os.listdir(dataDirectory))

# Any results you write to the current directory are saved as output.        

In [ ]:
!rm -r ~/.keras
!mkdir ~/.keras
!mkdir ~/.keras/models
# not enough space for both
#!cp ../input/keras-pretrained-models/* ~/.keras/models/ 
#!cp ../input/vgg19/* ~/.keras/models
!cp ../input/keras-pretrained-models/*notop* ~/.keras/models/
!cp ../input/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/
#!cp ../input/keras-pretrained-models/resnet50* ~/.keras/models/

In [ ]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.models import Model
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras.layers.convolutional import *
from keras.callbacks import ModelCheckpoint
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from keras.models import model_from_json
import itertools
import matplotlib.pyplot as plt
import time
import pandas as pd
%matplotlib inline

In [ ]:
train_path = dataDirectory+'/train'
test_path  = dataDirectory+'/test'
print(os.listdir(train_path))
print(os.listdir(test_path))

In [ ]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest',
    validation_split=0.2) # set validation split

In [ ]:
selectedClasses = ['copepod_cyclopoid_oithona', 'copepod_calanoid_frillyAntennae', 'radiolarian_colony', 'copepod_calanoid_large_side_antennatucked', 'hydromedusae_narco_dark', 'hydromedusae_other', 'trichodesmium_puff', 'radiolarian_chain', 'trichodesmium_multiple', 'fish_larvae_leptocephali', 'ctenophore_cydippid_tentacles', 'copepod_other', 'hydromedusae_liriope', 'siphonophore_partial', 'tornaria_acorn_worm_larvae', 'chordate_type1', 'protist_star', 'siphonophore_calycophoran_rocketship_adult', 'fish_larvae_medium_body', 'polychaete', 'jellies_tentacles', 'artifacts', 'invertebrate_larvae_other_B', 'hydromedusae_typeD_bell_and_tentacles', 'pteropod_triangle', 'tunicate_salp', 'hydromedusae_solmundella', 'tunicate_doliolid', 'appendicularian_fritillaridae', 'siphonophore_physonect', 'hydromedusae_bell_and_tentacles', 'fish_larvae_thin_body', 'ctenophore_cestid', 'fish_larvae_deep_body', 'artifacts_edge', 'copepod_calanoid_small_longantennae', 'crustacean_other', 'hydromedusae_partial_dark', 'hydromedusae_typeD', 'unknown_blobs_and_smudges', 'protist_noctiluca', 'siphonophore_physonect_young', 'hydromedusae_aglaura', 'copepod_calanoid_octomoms', 'detritus_filamentous', 'hydromedusae_typeF', 'siphonophore_other_parts', 'unknown_unclassified', 'detritus_blob', 'echinoderm_larva_pluteus_early', 'siphonophore_calycophoran_rocketship_young', 'appendicularian_slight_curve', 'appendicularian_s_shape', 'hydromedusae_typeE', 'acantharia_protist_big_center', 'echinoderm_larva_pluteus_typeC', 'trochophore_larvae', 'protist_dark_center', 'echinoderm_seacucumber_auricularia_larva', 'unknown_sticks', 'tunicate_salp_chains', 'invertebrate_larvae_other_A', 'pteropod_theco_dev_seq', 'amphipods', 'acantharia_protist_halo', 'acantharia_protist', 'echinoderm_larva_pluteus_brittlestar', 'chaetognath_other', 'chaetognath_sagitta', 'trichodesmium_tuft', 'hydromedusae_haliscera', 'fish_larvae_myctophids', 'decapods', 'trichodesmium_bowtie', 'tunicate_partial', 'hydromedusae_shapeA', 'heteropod', 'fish_larvae_very_thin_body', 'hydromedusae_solmaris', 'hydromedusae_haliscera_small_sideview', 'echinopluteus', 'pteropod_butterfly', 'euphausiids', 'echinoderm_larva_seastar_brachiolaria', 'ctenophore_lobate', 'shrimp-like_other', 'echinoderm_larva_pluteus_urchin', 'copepod_calanoid', 'protist_other', 'siphonophore_calycophoran_sphaeronectes', 'ephyra', 'copepod_calanoid_flatheads', 'siphonophore_calycophoran_sphaeronectes_young', 'siphonophore_calycophoran_sphaeronectes_stem', 'hydromedusae_narcomedusae', 'diatom_chain_tube', 'chaetognath_non_sagitta', 'shrimp_sergestidae', 'siphonophore_calycophoran_abylidae', 'copepod_cyclopoid_copilia', 'echinoderm_larva_seastar_bipinnaria', 'hydromedusae_shapeA_sideview_small', 'shrimp_zoea', 'fecal_pellet', 'shrimp_caridean', 'ctenophore_cydippid_no_tentacles', 'stomatopod', 'hydromedusae_sideview_big', 'copepod_calanoid_eggs', 'diatom_chain_string', 'tunicate_doliolid_nurse', 'hydromedusae_narco_young', 'copepod_calanoid_large', 'appendicularian_straight', 'hydromedusae_shapeB', 'hydromedusae_h15', 'copepod_calanoid_eucalanus', 'euphausiids_young', 'detritus_other', 'protist_fuzzy_olive', 'copepod_cyclopoid_oithona_eggs']

In [ ]:
batchSize=32


train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batchSize,
    classes=selectedClasses,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_path, # same directory as training data
    target_size=(224, 224),
    batch_size=batchSize,
    classes=selectedClasses,
    subset='validation') # set as validation data

test_generator = ImageDataGenerator().flow_from_directory(
    test_path, 
    target_size=(224,224), 
    classes=selectedClasses,
    shuffle= False,
    batch_size = batchSize)# set as test data

In [ ]:
print ("In train_generator ")
for cls in range(len (train_generator.class_indices)):
    print(selectedClasses[cls],":\t",list(train_generator.classes).count(cls))
print ("") 

print ("In validation_generator ")
for cls in range(len (validation_generator.class_indices)):
    print(selectedClasses[cls],":\t",list(validation_generator.classes).count(cls))
print ("") 

print ("In test_generator ")
for cls in range(len (test_generator.class_indices)):
    print(selectedClasses[cls],":\t",list(test_generator.classes).count(cls))

In [ ]:
#plots images with labels within jupyter notebook
def plots(ims, figsize = (22,22), rows=4, interp=False, titles=None, maxNum = 9):
    if type(ims[0] is np.ndarray):
        ims = np.array(ims).astype(np.uint8)
        if(ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
           
    f = plt.figure(figsize=figsize)
    #cols = len(ims) //rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    cols = maxNum // rows if maxNum % 2 == 0 else maxNum//rows + 1
    #for i in range(len(ims)):
    for i in range(maxNum):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=20)
        plt.imshow(ims[i], interpolation = None if interp else 'none')   

In [ ]:
train_generator.reset()
imgs, labels = train_generator.next()

#print(labels)

labelNames=[]
labelIndices=[np.where(r==1)[0][0] for r in labels]
#print(labelIndices)

for ind in labelIndices:
    for labelName,labelIndex in train_generator.class_indices.items():
        if labelIndex == ind:
            #print (labelName)
            labelNames.append(labelName)

#labels

In [ ]:
plots(imgs, rows=4, titles = labelNames, maxNum=8)

In [ ]:
#InceptionV3

base_model = InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(224, 224,3))
base_model.trainable = False

x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dropout(0.5)(x)
# and a sofymax/logistic layer -- we have 6 classes
predictions = Dense(len(selectedClasses), activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


model.summary()

In [ ]:
#Atutomatic rename with epoch number and val accuracy:
#filepath="checkpoints/weights-improvement-epeoch-{epoch:02d}-val_acc-{val_acc:.2f}.hdf5"


 
modelName= "InceptionTutorial"
#save the best weights over the same file with the model name

#filepath="checkpoints/"+modelName+"_bestweights.hdf5"
filepath=modelName+"_bestweights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
stepsPerEpoch= (train_generator.samples+ (batchSize-1)) // batchSize
print("stepsPerEpoch: ", stepsPerEpoch)

validationSteps=(validation_generator.samples+ (batchSize-1)) // batchSize
print("validationSteps: ", validationSteps)


#validationSteps=(test_generator.samples+ (batchSize-1)) // batchSize
#print("validationSteps: ", validationSteps)

In [ ]:
train_generator.reset()
validation_generator.reset()

# Fit the model
history = model.fit_generator(
    train_generator, 
    validation_data = validation_generator,
    epochs = 3,
    steps_per_epoch = stepsPerEpoch,
    validation_steps= validationSteps,
    callbacks=callbacks_list,
    verbose=1)

In [ ]:
# list all data in history
print(history.history.keys())

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
timestr = time.strftime("%Y%m%d_%H%M%S")

# serialize model to JSON
model_json = model.to_json()
with open(timestr+"_"+modelName+"_MODEL_3"+".json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(timestr+"_"+modelName+"_3_LAST_WEIGHTS_"+".h5")